In [1]:
import sys
import pandas as pd
import numpy as np
from sklearn import preprocessing
from sklearn.impute import SimpleImputer
pd.set_option('display.max_columns', None)
# pd.set_option('display.max_rows', None)

In [2]:
listings = pd.read_csv("data/listings.csv.gz")
reviews = pd.read_csv("data/reviews.csv.gz")

In [3]:
long_content_cols = ["description", "neighborhood_overview", "picture_url", 
                     "host_url", "host_about", "host_thumbnail_url", "host_picture_url"]
listings.drop(long_content_cols, axis=1).head()

,id,listing_url,scrape_id,last_scraped,name,host_id,host_name,host_since,host_location,host_response_time,host_response_rate,host_acceptance_rate,host_is_superhost,host_neighbourhood,host_listings_count,host_total_listings_count,host_verifications,host_has_profile_pic,host_identity_verified,neighbourhood,neighbourhood_cleansed,neighbourhood_group_cleansed,latitude,longitude,property_type,room_type,accommodates,bathrooms,bathrooms_text,bedrooms,beds,amenities,price,minimum_nights,maximum_nights,minimum_minimum_nights,maximum_minimum_nights,minimum_maximum_nights,maximum_maximum_nights,minimum_nights_avg_ntm,maximum_nights_avg_ntm,calendar_updated,has_availability,availability_30,availability_60,availability_90,availability_365,calendar_last_scraped,number_of_reviews,number_of_reviews_ltm,number_of_reviews_l30d,first_review,last_review,review_scores_rating,review_scores_accuracy,review_scores_cleanliness,review_scores_checkin,review_scores_communication,review_scores_location,review_scores_value,license,instant_bookable,calculated_host_listings_count,calculated_host_listings_count_entire_homes,calculated_host_listings_count_private_rooms,calculated_host_listings_count_shared_rooms,reviews_per_month
0,5570,https://www.airbnb.com/rooms/5570,20201223070513,2020-12-23,Ocean front condo on the sand,8435,Jef Karchin,2009-02-23,"San Diego, California, United States",within a few hours,100%,100%,f,Mission Beach,3.0,3.0,"['email', 'phone', 'kba']",t,t,"San Diego, California, United States",Mission Bay,NaN,32.78430,-117.25258,Entire condominium,Entire home/apt,6,NaN,1 bath,2.0,4.0,"[""Iron"", ""Elevator"", ""Stove"", ""Microwave"", ""Di...","$2,050.00",3,365,3.0,3.0,365.0,365.0,3.0,365.0,NaN,t,6,21,44,207,2020-12-23,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,f,3,3,0,0,NaN
1,29967,https://www.airbnb.com/rooms/29967,20201223070513,2020-12-23,"Great home, 10 min walk to Beach",129123,Michael,2010-05-21,"San Diego, California, United States",within an hour,100%,50%,t,Pacific Beach,6.0,6.0,"['email', 'phone', 'reviews', 'kba', 'work_ema...",t,t,NaN,Pacific Beach,NaN,32.80724,-117.25630,Entire bungalow,Entire home/apt,8,NaN,2 baths,2.0,3.0,"[""Iron"", ""Hot water"", ""Heating"", ""Kitchen"", ""W...",$261.00,4,365,4.0,4.0,365.0,365.0,4.0,365.0,NaN,t,6,6,6,213,2020-12-23,62,2,1,2010-07-09,2020-11-29,98.0,10.0,10.0,10.0,10.0,10.0,10.0,NaN,f,5,5,0,0,0.49
2,38245,https://www.airbnb.com/rooms/38245,20201223070513,2020-12-23,Point Loma: Den downstairs,164137,Melinda,2010-07-12,"San Diego, California, United States",within a day,100%,86%,f,Loma Portal,3.0,3.0,"['email', 'phone', 'reviews', 'kba']",t,t,"San Diego, California, United States",Roseville,NaN,32.74202,-117.21870,Private room in house,Private room,1,NaN,1 shared bath,1.0,1.0,"[""Iron"", ""Hot water"", ""Dishes and silverware"",...",$74.00,1,21,1.0,1.0,21.0,21.0,1.0,21.0,NaN,t,24,54,84,359,2020-12-23,143,0,0,2010-09-09,2019-10-20,86.0,9.0,9.0,10.0,10.0,9.0,9.0,NaN,f,3,0,3,0,1.14
3,54001,https://www.airbnb.com/rooms/54001,20201223070513,2020-12-23,"La Jolla 2 Bdr Cottage: 1 Qn; 2Twns, Blks 2 Ocn",252692,Marsha,2010-10-04,"San Diego, California, United States",within an hour,100%,75%,t,La Jolla,5.0,5.0,"['email', 'phone', 'reviews', 'kba']",t,t,"La Jolla, California, United States",La Jolla,NaN,32.81301,-117.26856,Entire guesthouse,Entire home/apt,3,NaN,1 bath,2.0,5.0,"[""Luggage dropoff allowed"", ""Single level home...",$110.00,3,365,3.0,30.0,18.0,1125.0,23.0,1079.9,NaN,t,0,0,18,99,2020-12-23,244,30,5,2011-01-04,2020-12-22,99.0,10.0,10.0,10.0,10.0,10.0,10.0,NaN,f,2,1,1,0,2.01
4,62274,https://www.airbnb.com/rooms/62274,20201223070513,2020-12-23,"charming, colorful, close to beach",302986,Isabel,2010-11-28,"San Diego, California, United States",within an hour,92%,95%,t,Pacific Beach,2.0,2.0,"['email', 'phone', 'reviews', 'kba']",t,t,"San Diego, California, United States",Pacific Beach,NaN,32.80734,-117.24243,Entire guesthouse,Entire home/apt,2,NaN,1 bath,1.0,1.0,"[""Luggage dropoff allowed"",

In [4]:
exclude_list = [] 
for col in listings.columns:
    value_cnts = len(listings[col].unique())
    if value_cnts < 2:
        exclude_list.append(col)

# too many null values 
is_null_cnt = listings.isnull().sum()
exclude_list += list(is_null_cnt[is_null_cnt > listings.shape[0]/4].index)
exclude_list = list(set(exclude_list))
print(exclude_list)


# no for analysis
exclude_list += ["scrape_id", "host_id"]

# get rid of columns
listings = listings.drop(exclude_list ,axis=1)

['neighborhood_overview', 'bathrooms', 'neighbourhood', 'neighbourhood_group_cleansed', 'host_about', 'calendar_updated', 'scrape_id', 'license']


In [5]:
df_numeric = listings.select_dtypes(exclude=['object'])

# via data inspections
useful_cat_varibles = ["neighbourhood_cleansed", "host_identity_verified", "host_neighbourhood"]
useful_cat_varibles += ["property_type", "room_type"]
useful_cat_varibles += ["host_response_time"]
df_cat = listings[useful_cat_varibles]

description_cols = ["description", "host_about"]
ser_amenities = listings.amenities

# format parsing
Y = listings.price.str.replace("$","").str.replace(",","").astype(float)

In [ ]:
df_all = pd.concat([df_numeric, df_cat], axis=1)
df_all["n_amenities"] = ser_amenities.apply(lambda x: len(eval(x)))
df_all["Y"] = Y

In [ ]:
#seperate training and testing data





In [18]:
#ref: https://stackoverflow.com/questions/21057621/sklearn-labelencoder-with-never-seen-before-values

In [15]:
le = preprocessing.LabelEncoder()
le.fit(df_cat["neighbourhood_cleansed"])
le_map = dict(zip(le.classes_, le.transform(le.classes_)))
others_idx = len(le.classes_) - 1 + 1

In [17]:
df[your_col] = df[your_col].apply(lambda x: le_map.get(x, others_idx))

102

In [81]:
# encode category variable into numeric one
le_dict = {}
categorical_feature = []
for col in df_cat.columns:
    le = preprocessing.LabelEncoder()
    le.fit(df_cat[col])
    le_dict[col] = le
    encoded_fts = le.transform(df_cat[col])
    df_cat[col] = encoded_fts
    categorical_feature.append(col)

/Users/lucaslee/anaconda3/envs/lucas_test/lib/python3.6/site-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':


In [72]:
df_all = pd.concat([df_numeric, df_cat], axis=1)
df_all["n_amenities"] = ser_amenities.apply(lambda x: len(eval(x)))

In [73]:
varibles_na_cnt = df_all.isnull().sum()
varibles_na_cnt[varibles_na_cnt > 0]

host_listings_count               3
host_total_listings_count         3
bedrooms                        997
beds                             50
minimum_minimum_nights            1
maximum_minimum_nights            1
minimum_maximum_nights            1
maximum_maximum_nights            1
minimum_nights_avg_ntm            1
maximum_nights_avg_ntm            1
review_scores_rating           1482
review_scores_accuracy         1525
review_scores_cleanliness      1524
review_scores_checkin          1527
review_scores_communication    1525
review_scores_location         1527
review_scores_value            1527
reviews_per_month              1441
dtype: int64

In [75]:
imputer_dict = {}
for col in varibles_na_cnt[varibles_na_cnt > 0].index:
    if "review" in col:
        value_to_fill = 0
    else:
        most_freq = df_all[col].value_counts().index[0]
        value_to_fill = most_freq    
    df_all[col].fillna(value_to_fill, inplace=True)
    imputer_dict[col] = value_to_fill


In [79]:
df_al

(9249, 42)

In [129]:
df_all.head()

,id,host_listings_count,host_total_listings_count,latitude,longitude,accommodates,bedrooms,beds,minimum_nights_x,maximum_nights_x,...,date,available,price,minimum_nights_y,maximum_nights_y,year,month,day,dayofyear,weekday
0,5570,3.0,3.0,32.7843,-117.25258,6,2.0,4.0,3,365,...,2020-12-23,f,2050.0,3.0,365.0,2020,12,23,358,2
1,5570,3.0,3.0,32.7843,-117.25258,6,2.0,4.0,3,365,...,2020-12-24,f,2050.0,3.0,365.0,2020,12,24,359,3
2,5570,3.0,3.0,32.7843,-117.25258,6,2.0,4.0,3,365,...,2020-12-25,f,2050.0,3.0,365.0,2020,12,25,360,4
3,5570,3.0,3.0,32.7843,-117.25258,6,2.0,4.0,3,365,...,2020-12-26,f,2050.0,3.0,365.0,2020,12,26,361,5
4,5570,3.0,3.0,32.7843,-117.25258,6,2.0,4.0,3,365,...,2020-12-27,f,2050.0,3.0,365.0,2020,12,27,362,6


In [76]:
reviews

,listing_id,id,date,reviewer_id,reviewer_name,comments
0,29967,62788,2010-07-09,151260,Debbie,When I booked our stay in San Diego at Dennis ...
1,29967,64568,2010-07-14,141552,Eric,This was my first experience with using airbnb...
2,29967,67502,2010-07-22,141591,David,We found the house to be very accommodating--e...
3,29967,70466,2010-07-29,125982,Anders,As advertised and more. Dennis was very helpfu...
4,29967,74876,2010-08-07,29835,Miyoko,We had a great time in San Diego. Denis' house...
...,...,...,...,...,...,...
443411,46995597,715948454,2020-12-16,346790349,Mike,"This is a great place with modern design, extr..."
443412,46995597,716097352,2020-12-17,370055591,Hanna,Such a amazing place! It makes me feel like ho...
443413,46995597,716330508,2020-12-18,299951196,Trina,Booked this place last minute and was able to ...
443414,46995597,716579344,2020-12-19,74213915,Colby,We had a great time staying at this house. It ...


In [75]:
reviews[reviews.listing_id == 29967].comments

0     When I booked our stay in San Diego at Dennis ...
1     This was my first experience with using airbnb...
2     We found the house to be very accommodating--e...
3     As advertised and more. Dennis was very helpfu...
4     We had a great time in San Diego. Denis' house...
                            ...                        
57    Michael’s place is great, very good location c...
58    Michaels home is spacious and clean. Was the p...
59    A very good neighborhood and house. The landlo...
60    Michael/Dennies were great hosts! Responded ve...
61          Great Location and nice for a small family.
Name: comments, Length: 62, dtype: object

In [ ]:
review_counts
review_senmentic_counts
